# Volumes

We’ve said that pods are similar to logical hosts where processes running inside them share resources such as CPU, RAM, network interfaces, and others. One would expect the processes to also share disks, but that’s not the case. You’ll remember that **each container in a pod has its own isolated filesystem**, because the file-system comes from the container’s image.

Every new container starts off with the exact set of files that was added to the image at build time. Combine this with the fact that containers in a pod get restarted (either because the process died or because the liveness probe signaled to Kubernetes that the container wasn’t healthy anymore) and you’ll realize that the new container will not see anything that was written to the filesystem by the previous container, even though the newly started container runs in the same pod.

In certain scenarios you want the new container to continue where the last one finished, such as when restarting a process on a physical machine. You may not need (or want) the whole filesystem to be persisted, but you do want to preserve the directories that hold actual data.

Kubernetes provides this by defining storage volumes. They aren’t top-level resources like pods, but are instead **defined as a part of a pod and share the same lifecycle as the pod**. This means a volume is created when the pod is started and is destroyed when the pod is deleted. Because of this, a volume’s contents will persist across container restarts. After a container is restarted, the new container can see all the files that were written to the volume by the previous container. Also, if a pod contains multiple containers, the volume can be used by all of them at once.

## Introducing volumes

Kubernetes volumes are a component of a pod and are thus defined in the pod’s specification—much like containers. They aren’t a standalone Kubernetes object and cannot be created or deleted on their own. A volume is available to all containers in the pod, but it must be mounted in each container that needs to access it. In each container, you can mount the volume in any location of its filesystem.

### Explaining volumes in an example

Imagine you have a pod with three containers (shown in figure 6.1). One container runs a web server that serves HTML pages from the /var/htdocs directory and stores the access log to /var/logs. The second container runs an agent that creates HTML files and stores them in /var/html. The third container processes the logs it finds in the /var/logs directory (rotates them, compresses them, analyzes them, or whatever).

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/06fig01.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_6-1.png" class="medium-zoom-image">



Each container has a nicely defined single responsibility, but on its own each container wouldn’t be of much use. Creating a pod with these three containers without them sharing disk storage doesn’t make any sense, because the content generator would write the generated HTML files inside its own container and the web server couldn’t access those files, as it runs in a separate isolated container. Instead, it would serve an empty directory or whatever you put in the /var/htdocs directory in its container image. Similarly, the log rotator would never have anything to do, because its /var/logs directory would always remain empty with nothing writing logs there. A pod with these three containers and no volumes basically does nothing.

But if you add two volumes to the pod and mount them at appropriate paths inside the three containers, as shown in figure 6.2, you’ve created a system that’s much more than the sum of its parts. Linux allows you to mount a filesystem at arbitrary locations in the file tree. When you do that, the contents of the mounted filesystem are accessible in the directory it’s mounted into. By mounting the same volume into two containers, they can operate on the same files. In your case, you’re mounting two volumes in three containers. By doing this, your three containers can work together and do something useful. Let me explain how.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/06fig02.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_6-2.png" class="medium-zoom-image">

First, the pod has a volume called publicHtml. This volume is mounted in the WebServer container at /var/htdocs, because that’s the directory the web server serves files from. The same volume is also mounted in the ContentAgent container, but at /var/html, because that’s where the agent writes the files to. By mounting this single volume like that, the web server will now serve the content generated by the content agent.

Similarly, the pod also has a volume called logVol for storing logs. This volume is mounted at /var/logs in both the WebServer and the LogRotator containers. Note that it isn’t mounted in the ContentAgent container. The container cannot access its files, even though the container and the volume are part of the same pod. It’s not enough to define a volume in the pod; you need to define a VolumeMount inside the container’s spec also, if you want the container to be able to access it.

The two volumes in this example can both initially be empty, so you can use a type of volume called emptyDir. Kubernetes also supports other types of volumes that are either populated during initialization of the volume from an external source, or an existing directory is mounted inside the volume. This process of populating or mounting a volume is performed before the pod’s containers are started.

A volume is bound to the lifecycle of a pod and will stay in existence only while the pod exists, but depending on the volume type, the volume’s files may remain intact even after the pod and volume disappear, and can later be mounted into a new volume. Let’s see what types of volumes exist.

### Introducing available volume types

A wide variety of volume types is available. Several are generic, while others are specific to the actual storage technologies used underneath. Don’t worry if you’ve never heard of those technologies — I hadn’t heard of at least half of them. You’ll probably only use volume types for the technologies you already know and use. Here’s a list of several of the available volume types:

- emptyDir—A simple empty directory used for storing transient data.
- hostPath—Used for mounting directories from the worker node’s filesystem into the pod.
- gitRepo—A volume initialized by checking out the contents of a Git repository.
- nfs—An NFS share mounted into the pod.
- gcePersistentDisk (Google Compute Engine Persistent Disk), awsElasticBlockStore (Amazon Web Services Elastic Block Store Volume), azureDisk (Microsoft Azure Disk Volume)—Used for mounting cloud provider-specific storage.
- cinder, cephfs, iscsi, flocker, glusterfs, quobyte, rbd, flexVolume, vsphere-Volume, photonPersistentDisk, scaleIO—Used for mounting other types of network storage.
- configMap, secret, downwardAPI—Special types of volumes used to expose certain Kubernetes resources and cluster information to the pod.
- persistentVolumeClaim—A way to use a pre- or dynamically provisioned persistent storage. (We’ll talk about them in the last section of this chapter.)

These volume types serve various purposes. You’ll learn about some of them in the following sections. Special types of volumes (secret, downwardAPI, configMap) are covered in the next two chapters, because they aren’t used for storing data, but for exposing Kubernetes metadata to apps running in the pod.

A single pod can use multiple volumes of different types at the same time, and, as we’ve mentioned before, each of the pod’s containers can either have the volume mounted or not.

## Using volumes to share data between containers
Although a volume can prove useful even when used by a single container, let’s first focus on how it’s used for sharing data between multiple containers in a pod.

### Using an emptyDir volume
The simplest volume type is the emptyDir volume, so let’s look at it in the first example of how to define a volume in a pod. As the name suggests, the volume starts out as an empty directory. The app running inside the pod can then write any files it needs to it. Because the volume’s lifetime is tied to that of the pod, the volume’s contents are lost when the pod is deleted.

An emptyDir volume is **especially useful for sharing files between containers running in the same pod**. But it can also be used by a single container for when a container needs to write data to disk temporarily, such as when performing a sort operation on a large dataset, which can’t fit into the available memory. The data could also be written to the container’s filesystem itself (remember the top read-write layer in a container?), but subtle differences exist between the two options. A container’s filesystem may not even be writable (we’ll talk about this toward the end of the book), so writing to a mounted volume might be the only option.

**Using an emptyDir volume in a pod**

Let’s revisit the previous example where a web server, a content agent, and a log rotator share two volumes, but let’s simplify a bit. You’ll build a pod with only the web server container and the content agent and a single volume for the HTML.

You’ll use Nginx as the web server and the UNIX fortune command to generate the HTML content. The fortune command prints out a random quote every time you run it. You’ll create a script that invokes the fortune command every 10 seconds and stores its output in index.html. You’ll find an existing Nginx image available on Docker Hub, but you’ll need to either create the fortune image yourself or use the one I’ve already built and pushed to Docker Hub under luksa/fortune. If you want a refresher on how to build Docker images, refer to the sidebar.

**Building the fortune container image**

Here’s how to build the image. Create a new directory called fortune and then inside it, create a `fortuneloop.sh` shell script with the following contents:

    #!/bin/bash
    trap "exit" SIGINT
    while :
    do
      echo $(date) Writing fortune to /var/htdocs/index.html
      /usr/games/fortune > /var/htdocs/index.html
      sleep 10
    done

Then, in the same directory, create a file called Dockerfile containing the following:

    FROM ubuntu:latest
    RUN apt-get update ; apt-get -y install fortune
    ADD fortuneloop.sh /bin/fortuneloop.sh
    ENTRYPOINT /bin/fortuneloop.sh


The image is based on the ubuntu:latest image, which doesn’t include the fortune binary by default. That’s why in the second line of the Dockerfile you install it with apt-get. After that, you add the fortuneloop.sh script to the image’s /bin folder. In the last line of the Dockerfile, you specify that the fortuneloop.sh script should be executed when the image is run.

After preparing both files, build and upload the image to Docker Hub with the following two commands (replace luksa with your own Docker Hub user ID):

    docker build -t leon11sj/fortune .
    docker push leon11sj/fortune

**Creating the pod**

Now that you have the two images required to run your pod, it’s time to create the pod manifest. Create a file called fortune-pod.yaml with the contents shown in the following listing.

```yml
apiVersion: v1
kind: Pod
metadata:
  name: fortune
spec:
  containers:
  - image: leon11sj/fortune
    name: html-generator
    volumeMounts:
    - name: html
      mountPath: /var/htdocs
  - image: nginx:alpine
    name: web-server
    volumeMounts:
    - name: html
      mountPath: /usr/share/nginx/html
      readOnly: true
    ports:
    - containerPort: 80
      protocol: TCP
  volumes:
  - name: html
    emptyDir: {}
```

The pod contains two containers and a single volume that’s mounted in both of them, yet at different paths. When the html-generator container starts, it starts writing the output of the fortune command to the /var/htdocs/index.html file every 10 seconds. Because the volume is mounted at /var/htdocs, the index.html file is written to the volume instead of the container’s top layer. As soon as the web-server container starts, it starts serving whatever HTML files are in the /usr/share/nginx/html directory (this is the default directory Nginx serves files from). Because you mounted the volume in that exact location, Nginx will serve the index.html file written there by the container running the fortune loop. The end effect is that a client sending an HTTP request to the pod on port 80 will receive the current fortune message as the response.

    kubectl apply -f ex01-fortune-pod.yaml

**Seeing the pod in action**

To see the fortune message, you need to enable access to the pod. You’ll do that by forwarding a port from your local machine to the pod:

    kubectl port-forward fortune 8080:80

> NOTE: As an exercise, you can also expose the pod through a service instead of using port forwarding.

Now you can access the Nginx server through port 8080 of your local machine. Use curl to do that:

    curl http://localhost:8080

If you wait a few seconds and send another request, you should receive a different message. By combining two containers, you created a simple app to see how a volume can glue together two containers and enhance what each of them does.

**Specifying the medium to use for the emptyDir**

The emptyDir you used as the volume was created on the actual disk of the worker node hosting your pod, so its performance depends on the type of the node’s disks. But you can tell Kubernetes to create the emptyDir on a tmpfs filesystem (in memory instead of on disk). To do this, set the emptyDir’s medium to Memory like this:

    volumes:
      - name: html
        emptyDir:
          medium: Memory

An emptyDir volume is the simplest type of volume, but other types build upon it. After the empty directory is created, they populate it with data. One such volume type is the gitRepo volume type, which we’ll introduce next.

### Using a Git repository as the starting point for a volume

A gitRepo volume is basically an emptyDir volume that gets populated by cloning a Git repository and checking out a specific revision when the pod is starting up (but before its containers are created). Figure 6.3 shows how this unfolds.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/06fig03_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_6-3.png" class="medium-zoom-image">

> NOTE: After the gitRepo volume is created, it isn’t kept in sync with the repo it’s referencing. The files in the volume will not be updated when you push additional commits to the Git repository. However, if your pod is managed by a ReplicationController, deleting the pod will result in a new pod being created and this new pod’s volume will then contain the latest commits.

For example, you can use a Git repository to store static HTML files of your website and create a pod containing a web server container and a gitRepo volume. Every time the pod is created, it pulls the latest version of your website and starts serving it. The only drawback to this is that you need to delete the pod every time you push changes to the gitRepo and want to start serving the new version of the website.

## Accessing files on the worker node’s filesystem

Most pods should be oblivious of their host node, so they shouldn’t access any files on the node’s filesystem. But certain system-level pods (remember, these will usually be managed by a DaemonSet) do need to either read the node’s files or use the node’s filesystem to access the node’s devices through the filesystem. Kubernetes makes this possible through a hostPath volume.

### Introducing the hostPath volume

A hostPath volume points to a specific file or directory on the node’s filesystem (see figure 6.4). Pods running on the same node and using the same path in their hostPath volume see the same files.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/06fig04_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_6-4.png" class="medium-zoom-image">

**hostPath volumes are the first type of persistent storage we’re introducing, because both the gitRepo and emptyDir volumes’ contents get deleted when a pod is torn down, whereas a hostPath volume’s contents don’t**. If a pod is deleted and the next pod uses a hostPath volume pointing to the same path on the host, the new pod will see whatever was left behind by the previous pod, but only if it’s scheduled to the same node as the first pod.

If you’re thinking of using a hostPath volume as the place to store a database’s data directory, think again. **Because the volume’s contents are stored on a specific node’s filesystem, when the database pod gets rescheduled to another node, it will no longer see the data**. This explains why it’s not a good idea to use a hostPath volume for regular pods, because it makes the pod sensitive to what node it’s scheduled to.

> TIP: Remember to use hostPath volumes only if you need to read or write system files on the node. Never use them to persist data across pods.

## Using persistent storage

When an application running in a pod needs to persist data to disk and have that same data available even when the pod is rescheduled to another node, you can’t use any of the volume types we’ve mentioned so far. Because this data needs to be accessible from any cluster node, it must be stored on some type of network-attached storage (NAS).

To learn about volumes that allow persisting data, you’ll create a pod that will run the MongoDB document-oriented NoSQL database. Running a database pod without a volume or with a non-persistent volume doesn’t make sense, except for testing purposes, so you’ll add an appropriate type of volume to the pod and mount it in the MongoDB container.

### Using a GCE Persistent Disk in a pod volume

If you’ve been running these examples on Google Kubernetes Engine, which runs your cluster nodes on Google Compute Engine (GCE), you’ll use a GCE Persistent Disk as your underlying storage mechanism.

In the early versions, Kubernetes didn’t provision the underlying storage automatically—you had to do that manually. Automatic provisioning is now possible, and you’ll learn about it later in the chapter, but first, you’ll start by provisioning the storage manually. It will give you a chance to learn exactly what’s going on underneath.

**Creating a GCE Persistent Disk**

You’ll start by creating the GCE persistent disk first. You need to create it in the same zone as your Kubernetes cluster. If you don’t remember what zone you created the cluster in, you can see it by listing your Kubernetes clusters with the gcloud command like this:

    gcloud container clusters list

This shows you’ve created your cluster in zone europe-west1-b, so you need to create the GCE persistent disk in the same zone as well. You create the disk like this:

    gcloud compute disks create mongodb --size=10GiB --zone=europe-west3-a

    gcloud compute disks list

This command creates a 10 GiB large GCE persistent disk called mongodb. You can ignore the warning about the disk size, because you don’t care about the disk’s performance for the tests you’re about to run.

**Creating a pod using a gcePersistentDisk volume**

Now that you have your physical storage properly set up, you can use it in a volume inside your MongoDB pod. You’re going to prepare the YAML for the pod, which is shown in the following listing.

```yml
apiVersion: v1
kind: Pod
metadata:
  name: mongodb
spec:
  volumes:
  - name: mongodb-data
    gcePersistentDisk:
      pdName: mongodb
      fsType: ext4
  containers:
  - image: mongo
    name: mongodb
    volumeMounts:
    - name: mongodb-data
      mountPath: /data/db
    ports:
    - containerPort: 27017
      protocol: TCP
```

The pod contains a single container and a single volume backed by the GCE Persistent Disk you’ve created (as shown in figure 6.5). You’re mounting the volume inside the container at /data/db, because that’s where MongoDB stores its data.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/06fig05_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_6-5.png" class="medium-zoom-image">

    kubectl apply -f ex02-mongodb-pod-gcepd.yaml

**Writing data to the persistent storage by adding documents to your MongoDB database**

Now that you’ve created the pod and the container has been started, you can run the MongoDB shell inside the container and use it to write some data to the data store.

You’ll run the shell as shown in the following listing.

    kubectl exec -it mongodb -- mongo

MongoDB allows storing JSON documents, so you’ll store one to see if it’s stored persistently and can be retrieved after the pod is re-created. Insert a new JSON document with the following commands:

    use mystore
    db.foo.insert({name:'foo'})

You’ve inserted a simple JSON document with a single property (name: 'foo'). Now, use the find() command to see the document you inserted:

     db.foo.find()

There it is. The document should be stored in your GCE persistent disk now.

**Re-creating the pod and verifying that it can read the data persisted by the previous pod**

You can now exit the mongodb shell (type exit and press Enter), and then delete the pod and recreate it:

    kubectl delete pod mongodb
    kubectl apply -f ex02-mongodb-pod-gcepd.yaml

The new pod uses the exact same GCE persistent disk as the previous pod, so the MongoDB container running inside it should see the exact same data, even if the pod is scheduled to a different node.

> You can see what node a pod is scheduled to by running `kubectl get po -o wide`.

Once the container is up, you can again run the MongoDB shell and check to see if the document you stored earlier can still be retrieved, as shown in the following listing.

    kubectl exec -it mongodb -- mongo

    use mystore
    db.foo.find()




As expected, the data is still there, even though you deleted the pod and re-created it. This confirms you can use a GCE persistent disk to persist data across multiple pod instances.

You’re done playing with the MongoDB pod, so go ahead and delete it again, but hold off on deleting the underlying GCE persistent disk. You’ll use it again later in the chapter.

    kubectl delete pod mongodb

### Using other types of volumes with underlying persistent storage

The reason you created the GCE Persistent Disk volume is because your Kubernetes cluster runs on Google Kubernetes Engine. When you run your cluster elsewhere, you should use other types of volumes, depending on the underlying infrastructure.

If your Kubernetes cluster is running on Amazon’s AWS EC2, for example, you can use an awsElasticBlockStore volume to provide persistent storage for your pods. If your cluster runs on Microsoft Azure, you can use the azureFile or the azureDisk volume. We won’t go into detail on how to do that here, but it’s virtually the same as in the previous example. First, you need to create the actual underlying storage, and then set the appropriate properties in the volume definition.

**Using an AWS Elastic Block Store volume**

For example, to use an AWS elastic block store instead of the GCE Persistent Disk, you’d only need to change the volume definition as shown in the following listing (see those lines printed in bold).

An [awsElasticBlockStore](https://kubernetes.io/docs/concepts/storage/volumes/#awselasticblockstore) volume mounts an Amazon Web Services (AWS) EBS Volume into your Pod. Unlike emptyDir, which is erased when a Pod is removed, the contents of an EBS volume are preserved and the volume is merely unmounted. **This means that an EBS volume can be pre-populated with data, and that data can be "handed off" between Pods**.

    kubectl label node ip-172-31-36-56.eu-central-1.compute.internal ebs_volume=false --overwrite
    kubectl label node ip-172-31-43-116.eu-central-1.compute.internal ebs_volume=true --overwrite

> Caution: You must create an EBS volume using aws ec2 create-volume or the AWS API before you can use it.

https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ebs-creating-volume.html

There are some restrictions when using an awsElasticBlockStore volume:
- the nodes on which Pods are running must be AWS EC2 instances
- those instances need to be in the same region and availability-zone as the EBS volume
- EBS only supports a single EC2 instance mounting a volume

    aws ec2 create-volume --availability-zone=eu-central-1b --size=2 --volume-type=gp2

Volume ID: `vol-049ae159432d59bdd`

> ex06-mongodb-pod-aws.yaml

```yml
apiVersion: v1
kind: Pod
metadata:
  name: mongodb-aws
spec:
  volumes:
  - name: mongodb-data
    awsElasticBlockStore:
      volumeID: vol-049ae159432d59bdd
      fsType: ext4
  nodeSelector:
    ebs_volume: "true"
  containers:
  - image: mongo
    name: mongodb
    volumeMounts:
    - name: mongodb-data
      mountPath: /data/db
    ports:
    - containerPort: 27017
      protocol: TCP

```



    kubectl apply -f ex06-mongodb-pod-aws.yaml    

Now that you’ve created the pod and the container has been started, you can run the MongoDB shell inside the container and use it to write some data to the data store.

You’ll run the shell as shown in the following listing.

    kubectl exec -it mongodb-aws -- mongo

MongoDB allows storing JSON documents, so you’ll store one to see if it’s stored persistently and can be retrieved after the pod is re-created. Insert a new JSON document with the following commands:

    use mystore
    db.foo.insert({name:'foo'})

You’ve inserted a simple JSON document with a single property (name: 'foo'). Now, use the find() command to see the document you inserted:

     db.foo.find()

You can now exit the mongodb shell (type exit and press Enter), and then delete the pod and recreate it:

    kubectl delete pod mongodb-aws
    kubectl apply -f ex06-mongodb-pod-aws.yaml  
    kubectl exec -it mongodb-aws -- mongo

    use mystore
    db.foo.find()

    kubectl delete pod mongodb-aws

**Using an NFS volume**

If your cluster is running on your own set of servers, you have a vast array of other supported options for mounting external storage inside your volume. For example, to mount a simple NFS share, you only need to specify the NFS server and the path exported by the server, as shown in the following listing.

**Using other storage technologies**

Having a pod’s volumes refer to the actual underlying infrastructure isn’t what Kubernetes is about, is it? For example, for a developer to have to specify the hostname of the NFS server feels wrong. And that’s not even the worst thing about it.

Including this type of infrastructure-related information into a pod definition means the pod definition is pretty much tied to a specific Kubernetes cluster. You can’t use the same pod definition in another one. That’s why using volumes like this isn’t the best way to attach persistent storage to your pods. You’ll learn how to improve on this in the next section.

## Decoupling pods from the underlying storage technology

All the persistent volume types we’ve explored so far **have required the developer of the pod to have knowledge of the actual network storage infrastructure available in the cluster**. For example, to create a NFS-backed volume, the developer has to know the actual server the NFS export is located on. This is against the basic idea of Kubernetes, which **aims to hide the actual infrastructure from both the application and its developer**, leaving them free from worrying about the specifics of the infrastructure and making apps portable across a wide array of cloud providers and on-premises datacenters.

Ideally, a developer deploying their apps on Kubernetes should never have to know what kind of storage technology is used underneath, the same way they don’t have to know what type of physical servers are being used to run their pods. Infrastructure-related dealings should be the sole domain of the cluster administrator.

When a developer needs a certain amount of persistent storage for their application, they can request it from Kubernetes, the same way they can request CPU, memory, and other resources when creating a pod. The system administrator can configure the cluster so it can give the apps what they request.

### Introducing PersistentVolumes and PersistentVolumeClaims

To enable apps to request storage in a Kubernetes cluster without having to deal with infrastructure specifics, two new resources were introduced. They are Persistent-Volumes and PersistentVolumeClaims. The names may be a bit misleading, because as you’ve seen in the previous few sections, even regular Kubernetes volumes can be used to store persistent data.

Using a PersistentVolume inside a pod is a little more complex than using a regular pod volume, so let’s illustrate how pods, PersistentVolumeClaims, PersistentVolumes, and the actual underlying storage relate to each other in figure 6.6.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/06fig06_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_6-6.png" class="medium-zoom-image">

Instead of the developer adding a technology-specific volume to their pod, it’s the cluster administrator who sets up the underlying storage and then registers it in Kubernetes by creating a PersistentVolume resource through the Kubernetes API server. When creating the PersistentVolume, the admin specifies its size and the access modes it supports.

When a cluster user needs to use persistent storage in one of their pods, they first create a PersistentVolumeClaim manifest, specifying the minimum size and the access mode they require. The user then submits the PersistentVolumeClaim manifest to the Kubernetes API server, and Kubernetes finds the appropriate PersistentVolume and binds the volume to the claim.

The PersistentVolumeClaim can then be used as one of the volumes inside a pod. Other users cannot use the same PersistentVolume until it has been released by deleting the bound PersistentVolumeClaim.

### Creating a PersistentVolume

Let’s revisit the MongoDB example, but unlike before, you won’t reference the GCE Persistent Disk in the pod directly. Instead, you’ll first assume the role of a cluster administrator and create a PersistentVolume backed by the GCE Persistent Disk. Then you’ll assume the role of the application developer and first claim the PersistentVolume and then use it inside your pod.

In section 6.4.1 you set up the physical storage by provisioning the GCE Persistent Disk, so you don’t need to do that again. All you need to do is create the Persistent-Volume resource in Kubernetes by preparing the manifest shown in the following listing and posting it to the API server.

```yml
apiVersion: v1
kind: PersistentVolume
metadata:
  name: mongodb-pv
spec:
  capacity:
    storage: 10Gi
  accessModes:
  - ReadWriteOnce
  - ReadOnlyMany
  persistentVolumeReclaimPolicy: Retain
  gcePersistentDisk:
    pdName: mongodb
    fsType: ext4
        
 ```

    kubectl apply -f ex03-mongodb-pv-gcepd.yaml

When creating a PersistentVolume, the administrator needs to tell Kubernetes what its capacity is and whether it can be read from and/or written to by a single node or by multiple nodes at the same time. They also need to tell Kubernetes what to do with the PersistentVolume when it’s released (when the PersistentVolumeClaim it’s bound to is deleted). And last, but certainly not least, they need to specify the type, location, and other properties of the actual storage this PersistentVolume is backed by. If you look closely, this last part is exactly the same as earlier, when you referenced the GCE Persistent Disk in the pod volume directly (shown again in the following listing).

    spec:
      volumes:
      - name: mongodb-data
        gcePersistentDisk:
          pdName: mongodb
          fsType: ext4
      ...

After you create the PersistentVolume with the kubectl create command, it should be ready to be claimed. See if it is by listing all PersistentVolumes:

    kubectl get pv

As expected, the PersistentVolume is shown as Available, because you haven’t yet created the PersistentVolumeClaim.

> NOTE: PersistentVolumes don’t belong to any namespace (see figure 6.7). They’re cluster-level resources like nodes.

<img alt="" src="https://drek4537l1klr.cloudfront.net/luksa/Figures/06fig07_alt.jpg" data-action="zoom" data-zoom-src="https://drek4537l1klr.cloudfront.net/luksa/HighResolutionFigures/figure_6-7.png" class="medium-zoom-image">

### Claiming a PersistentVolume by creating a PersistentVolumeClaim

Now let’s lay down our admin hats and put our developer hats back on. Say you need to deploy a pod that requires persistent storage. You’ll use the PersistentVolume you created earlier. But you can’t use it directly in the pod. You need to claim it first.

**Claiming a PersistentVolume is a completely separate process from creating a pod, because you want the same PersistentVolumeClaim to stay available even if the pod is rescheduled** (remember, rescheduling means the previous pod is deleted and a new one is created).

**Creating a PersistentVolumeClaim**

You’ll create the claim now. You need to prepare a PersistentVolumeClaim manifest like the one shown in the following listing and post it to the Kubernetes API through kubectl create.

```yml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: mongodb-pvc
spec:
  resources:
    requests:
      storage: 10Gi
  accessModes:
  - ReadWriteOnce
  storageClassName: ""
 ```

As soon as you create the claim, Kubernetes finds the appropriate PersistentVolume and binds it to the claim. The PersistentVolume’s capacity must be large enough to accommodate what the claim requests. Additionally, the volume’s access modes must include the access modes requested by the claim. In your case, the claim requests 1 GiB of storage and a ReadWriteOnce access mode. The PersistentVolume you created earlier matches those two requirements so it is bound to your claim. You can see this by inspecting the claim.

    kubectl apply -f ex04-mongodb-pvc.yaml

**Listing PersistentVolumeClaims**

List all PersistentVolumeClaims to see the state of your PVC:

    kubectl get pvc

The claim is shown as Bound to PersistentVolume mongodb-pv. Note the abbreviations used for the access modes:
- RWO - ReadWriteOnce—Only a single node can mount the volume for reading and writing.
- ROX - ReadOnlyMany—Multiple nodes can mount the volume for reading.
- RWX - ReadWriteMany—Multiple nodes can mount the volume for both reading and writing.

> RWO, ROX, and RWX pertain to the number of worker nodes that can use the volume at the same time, not to the number of pods!

**Listing PersistentVolumes**

You can also see that the PersistentVolume is now Bound and no longer Available by inspecting it with kubectl get:

     kubectl get pv

The PersistentVolume shows it’s bound to claim default/mongodb-pvc. The default part is the namespace the claim resides in (you created the claim in the default namespace). We’ve already said that PersistentVolume resources are cluster-scoped and thus cannot be created in a specific namespace, but PersistentVolumeClaims can only be created in a specific namespace. They can then only be used by pods in the same namespace.

### Using a PersistentVolumeClaim in a pod

The PersistentVolume is now yours to use. Nobody else can claim the same volume until you release it. To use it inside a pod, you need to reference the PersistentVolumeClaim by name inside the pod’s volume (yes, the PersistentVolumeClaim, not the PersistentVolume directly!), as shown in the following listing.

```yml
apiVersion: v1
kind: Pod
metadata:
  name: mongodb
spec:
  containers:
  - image: mongo
    name: mongodb
    volumeMounts:
    - name: mongodb-data
      mountPath: /data/db
    ports:
    - containerPort: 27017
      protocol: TCP
  volumes:
  - name: mongodb-data
    persistentVolumeClaim:
      claimName: mongodb-pvc
```

    kubectl apply -f ex05-mongodb-pod-pvc.yaml

Go ahead and create the pod. Now, check to see if the pod is indeed using the same PersistentVolume and its underlying GCE PD. You should see the data you stored earlier by running the MongoDB shell again, as shown in the following listing.

    kubectl exec -it mongodb -- mongo
    use mystore
    db.foo.find()

And there it is. You‘re able to retrieve the document you stored into MongoDB previously.

Examine figure 6.8, which shows both ways a pod can use a GCE Persistent Disk—directly or through a PersistentVolume and claim.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/06fig08_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_6-8.png" class="medium-zoom-image">

Consider how using this indirect method of obtaining storage from the infrastructure is much simpler for the application developer (or cluster user). Yes, it does require the additional steps of creating the PersistentVolume and the Persistent-VolumeClaim, but the developer doesn’t have to know anything about the actual storage technology used underneath.

Additionally, the same pod and claim manifests can now be used on many different Kubernetes clusters, because they don’t refer to anything infrastructure-specific. The claim states, “I need x amount of storage and I need to be able to read and write to it by a single client at once,” and then the pod references the claim by name in one of its volumes.

## Dynamic provisioning of PersistentVolumes

You’ve seen how using PersistentVolumes and PersistentVolumeClaims makes it easy to obtain persistent storage without the developer having to deal with the actual storage technology used underneath. But this still requires a cluster administrator to provision the actual storage up front. Luckily, Kubernetes can also perform this job automatically through dynamic provisioning of PersistentVolumes.

The cluster admin, instead of creating PersistentVolumes, can deploy a PersistentVolume provisioner and define one or more StorageClass objects to let users choose what type of PersistentVolume they want. The users can refer to the StorageClass in their PersistentVolumeClaims and the provisioner will take that into account when provisioning the persistent storage.

> NOTE: Similar to PersistentVolumes, StorageClass resources aren’t namespaced.

Kubernetes includes provisioners for the most popular cloud providers, so the administrator doesn’t always need to deploy a provisioner. But if Kubernetes is deployed on-premises, a custom provisioner needs to be deployed.

Instead of the administrator pre-provisioning a bunch of PersistentVolumes, they need to define one or two (or more) StorageClasses and let the system create a new PersistentVolume each time one is requested through a PersistentVolumeClaim. The great thing about this is that it’s impossible to run out of PersistentVolumes (obviously, you can run out of storage space).

### Defining the available storage types through StorageClass resources

Before a **user can create a PersistentVolumeClaim, which will result in a new Persistent-Volume being provisioned, an admin needs to create one or more StorageClass resources**. Let’s look at an example of one in the following listing.

ex09-storageclass-fast.yaml

```yml 
kind: StorageClass
apiVersion: storage.k8s.io/v1
metadata:
  name: fast
provisioner: kubernetes.io/aws-ebs
parameters:
  type: gp2
  fsType: ext4
```

    kubectl apply -f ex09-storageclass-fast.yaml

https://kubernetes.io/docs/concepts/storage/storage-classes/#aws-ebs

<ul><li><code>type</code>: <code>io1</code>, <code>gp2</code>, <code>sc1</code>, <code>st1</code>. See
<a href="https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/EBSVolumeTypes.html">AWS docs</a>
for details. Default: <code>gp2</code>.</li><li><code>zone</code> (Deprecated): AWS zone. If neither <code>zone</code> nor <code>zones</code> is specified, volumes are
generally round-robin-ed across all active zones where Kubernetes cluster
has a node. <code>zone</code> and <code>zones</code> parameters must not be used at the same time.</li><li><code>zones</code> (Deprecated): A comma separated list of AWS zone(s). If neither <code>zone</code> nor <code>zones</code>
is specified, volumes are generally round-robin-ed across all active zones
where Kubernetes cluster has a node. <code>zone</code> and <code>zones</code> parameters must not
be used at the same time.</li><li><code>iopsPerGB</code>: only for <code>io1</code> volumes. I/O operations per second per GiB. AWS
volume plugin multiplies this with size of requested volume to compute IOPS
of the volume and caps it at 20 000 IOPS (maximum supported by AWS, see
<a href="https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/EBSVolumeTypes.html">AWS docs</a>.
A string is expected here, i.e. <code>"10"</code>, not <code>10</code>.</li><li><code>fsType</code>: fsType that is supported by kubernetes. Default: <code>"ext4"</code>.</li><li><code>encrypted</code>: denotes whether the EBS volume should be encrypted or not.
Valid values are <code>"true"</code> or <code>"false"</code>. A string is expected here,
i.e. <code>"true"</code>, not <code>true</code>.</li><li><code>kmsKeyId</code>: optional. The full Amazon Resource Name of the key to use when
encrypting the volume. If none is supplied but <code>encrypted</code> is true, a key is
generated by AWS. See AWS docs for valid ARN value.</li></ul>

The StorageClass resource specifies which provisioner should be used for provisioning the PersistentVolume when a PersistentVolumeClaim requests this StorageClass. The parameters defined in the StorageClass definition are passed to the provisioner and are specific to each provisioner plugin.

    kubectl get storageclass

Beside the fast storage class, which you created yourself, a standard storage class exists and **is marked as default**. You’ll learn what that means in a moment. 

You’re going to use kubectl get to see more info about the standard storage class in a AWS cluster, as shown in the following listing.

    kubectl describe sc gp2

If you look closely toward the top of the listing, the storage class definition includes an annotation, which makes this the default storage class. The **default storage class is what’s used to dynamically provision a PersistentVolume if the PersistentVolumeClaim doesn’t explicitly say which storage class to use**.

### Requesting the storage class in a PersistentVolumeClaim

After the StorageClass resource is created, users can refer to the storage class by name in their PersistentVolumeClaims.

**Creating a PVC definition requesting a specific storage class**

You can modify your mongodb-pvc to use dynamic provisioning. The following listing shows the updated YAML definition of the PVC.

> You can create a PVC without specifying the storageClassName attribute and (on Google Kubernetes Engine) a Persistent Disk of type gp2 will be provisioned for you. Try this by creating a claim from the YAML in the following listing.

ex-07-mongodb-pvc-dp.yaml

```yml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: mongodb-pvc-fast 
spec:
  storageClassName: fast
  resources:
    requests:
      storage: 1Gi
  accessModes:
    - ReadWriteOnce

    
```

    kubectl apply -f ex-07-mongodb-pvc-dp.yaml

Apart from specifying the size and access modes, your PersistentVolumeClaim now also specifies the class of storage you want to use. When you create the claim, the PersistentVolume is created by the provisioner referenced in the fast StorageClass resource. The provisioner is used even if an existing manually provisioned PersistentVolume matches the PersistentVolumeClaim.

ex08-mongodb-pod-pvc.yaml

```yml
apiVersion: v1
kind: Pod
metadata:
  name: mongodb-fast 
spec:
  containers:
  - image: mongo
    name: mongodb
    volumeMounts:
    - name: mongodb-data
      mountPath: /data/db
    ports:
    - containerPort: 27017
      protocol: TCP
  volumes:
  - name: mongodb-data
    persistentVolumeClaim:
      claimName: mongodb-pvc-fast
```

    kubectl apply -f ex08-mongodb-pod-pvc.yaml

**Examining the created PVC and the dynamically provisioned PV**

Examining the created PVC and the dynamically provisioned PV

    kubectl get pvc

The VOLUME column shows the PersistentVolume that’s bound to this claim (the actual name is longer than what’s shown above). You can try listing PersistentVolumes now to see that a new PV has indeed been created automatically:

> Admin - non namespaced resource

     kubectl get pv

You can see the dynamically provisioned PersistentVolume. Its capacity and access modes are what you requested in the PVC. Its reclaim policy is Delete, which means the PersistentVolume will be deleted when the PVC is deleted. Beside the PV, the provisioner also provisioned the actual storage. 

**Understanding how to use storage classes**

The cluster admin can **create multiple storage classes with different performance or other characteristics**. The developer then decides which one is most appropriate for each claim they create.

The nice thing about StorageClasses is the fact that claims refer to them by name. The PVC definitions are therefore portable across different clusters, as long as the StorageClass names are the same across all of them.

**Understanding the complete picture of dynamic PersistentVolume provisioning**

To summarize, the best way to attach persistent storage to a pod is to only create the PVC (with an explicitly specified storageClassName if necessary) and the pod (which refers to the PVC by name). Everything else is taken care of by the dynamic PersistentVolume provisioner.

<img alt="" src="https://drek4537l1klr.cloudfront.net/luksa/Figures/06fig10_alt.jpg" data-action="zoom" data-zoom-src="https://drek4537l1klr.cloudfront.net/luksa/HighResolutionFigures/figure_6-10.png" class="medium-zoom-image">